In [1]:
import pandas as pd
%load_ext autoreload
%autoreload 2

In [2]:
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

In [3]:
from algorithm.kgvec2go_vectors_simple import run_method, load_gensim_model
from excludes.basic_excludes import (node_excl_yago_func, node_excl_extra,
                                     node_excl_owlthing_func, node_excl_wiki_func)
from loaders.loaders import *
from evaluation.split_dataset import split_on_seed_dataset
import json
import pickle
import os
from tqdm import tqdm

SEED = 44
SAMPLE_SIZE = 4
verbose = 1

lc_quad_train = load_on_path(ds_lc_quad_train_cleaned)
splitted_lcquad = split_on_seed_dataset(lc_quad_train, SAMPLE_SIZE, SEED)

cwd = "/home/kardosp/continuethelist/dbpedia_vectors/kgvec2go"

word2vec_path = os.path.join(cwd, 'model.kv')
wv_model = load_gensim_model(word2vec_path)


In [4]:
splitted_lcquad[0]

{'_id': '2413',
 'corrected_question': 'Starwood operates in which places?',
 'intermediary_question': 'What is the <address> of the <hotels> whose <operator> is <Starwood Hotels and Resorts Worldwide>?',
 'sparql_query': 'SELECT DISTINCT ?uri WHERE { ?x <http://dbpedia.org/property/operator> <http://dbpedia.org/resource/Starwood_Hotels_and_Resorts_Worldwide> . ?x <http://dbpedia.org/property/address> ?uri  . }',
 'sparql_template_id': 5,
 'result': ['181',
  'http://dbpedia.org/resource/California',
  'http://dbpedia.org/resource/San_Francisco',
  '2',
  'Calle San Fernando, 2',
  '1',
  'http://dbpedia.org/resource/Chennai',
  'http://dbpedia.org/resource/Tamil_Nadu',
  'Krakowskie Przedmiescie 42/44',
  '1900',
  'http://dbpedia.org/resource/Seattle',
  '811',
  '333',
  '210',
  'http://dbpedia.org/resource/Atlanta',
  '10600',
  'http://dbpedia.org/resource/Westminster,_Colorado',
  'Main Gulshan Avenue, Plot-01, Road 45, Gulshan-2, Dhaka 1212, Bangladesh',
  '211',
  'http://dbpe

In [11]:
missing_counter = 0
missing_nodes = []
all_nodes = 0
for i, record in tqdm(list(enumerate(splitted_lcquad))):
    for node in record["seed"]:
        all_nodes += 1
        if node not in wv_model:
            missing_counter += 1
            missing_nodes.append(node)

    for node in record["gold"]:
        all_nodes += 1
        if node not in wv_model:
            missing_counter += 1
            missing_nodes.append(node)

100%|██████████████████████████████████████| 250/250 [00:00<00:00, 22895.17it/s]


In [10]:
print("Missing:", missing_counter)
print("All:", all_nodes)
print(1-(missing_counter/all_nodes))

Missing: 417
All: 7779
0.9463941380640185


In [12]:
missing_nodes

['http://dbpedia.org/resource/Courtship',
 'http://dbpedia.org/resource/Intellect',
 'http://dbpedia.org/resource/Family_Memoir',
 'http://dbpedia.org/resource/Creative_class',
 'http://dbpedia.org/resource/Starr_(rapper)',
 "http://dbpedia.org/resource/Big_B'Z",
 'http://dbpedia.org/resource/Hynief',
 'http://dbpedia.org/resource/SkullGizzy(Gamer)',
 'http://dbpedia.org/resource/Nick_Haze',
 'http://dbpedia.org/resource/Richmond_Rab',
 'http://dbpedia.org/resource/Riq_Rose',
 'http://dbpedia.org/resource/Yung_Ja',
 'http://dbpedia.org/resource/Hajime_Kazu',
 'http://dbpedia.org/resource/Kaoru_Kurosaki',
 'http://dbpedia.org/resource/Robinson_Haruhara',
 "http://dbpedia.org/resource/Ulmus_×_hollandica_'Virens'",
 'http://dbpedia.org/resource/Alive_Day_Memories:_Home_from_Iraq',
 'http://dbpedia.org/resource/Siegfried_Geisler',
 'http://dbpedia.org/resource/Eckardt_Köppen',
 'http://dbpedia.org/resource/Michael_Bird_(politician)',
 'http://dbpedia.org/resource/Heinz_Bäumler',
 'http://d

In [5]:
with open("/home/kardosp/continuethelist/datasets/LC-QuAD_splitted/lcquad_splitted_seed44_samplesize4.pickle", "wb") as f:
    pickle.dump(splitted_lcquad, f)